In [1]:
!pip install adjustText

In [1]:
%matplotlib inline

from datetime import datetime

import geopandas as gp
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

from shapely.geometry import Point
from adjustText import adjust_text

In [10]:
def load_and_intersect(year, quarter, distrital_gdf):
    file_path = f'../input/{year}/pe_{year}_{quarter}.shp'  # Ruta del archivo de entrada
    gdf = gp.read_file(file_path)  # Carga el shapefile
    gdf = gdf.to_crs(distrital_gdf.crs)  # Asegura que los CRS sean compatibles

    # Realiza la intersección espacial
    distrital_data = gp.sjoin(gdf, distrital_gdf, how="inner", predicate="intersects")

    # Añade columnas de año y trimestre
    distrital_data['year'] = year
    distrital_data['quarter'] = quarter

    return distrital_data

In [11]:
# Carga el shapefile de los distritos de Lima
distrital_gdf = gp.read_file("../input/Shapefiles Lima Peru/Distritos/Distrital INEI 2023 geogpsperu SuyoPomalia.shp")

# Lista de años y trimestres
years = [2019, 2020, 2021, 2022]
quarters = [1, 2, 3, 4]

# Lista para almacenar los DataFrames
all_data = []


In [12]:
# Procesa cada año y trimestre
for year in years:
    for quarter in quarters:
        try:
            data = load_and_intersect(year, quarter, distrital_gdf)
            all_data.append(data)
        except Exception as e:
            print(f"Error processing {year} Q{quarter}: {e}")

In [13]:
# Combina todos los DataFrames
combined_data_1 = pd.concat(all_data, ignore_index=True)

In [16]:
combined_data_1.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 158448 entries, 0 to 158447
Data columns (total 19 columns):
 #   Column       Non-Null Count   Dtype   
---  ------       --------------   -----   
 0   quadkey      158448 non-null  object  
 1   avg_d_kbps   158448 non-null  int64   
 2   avg_u_kbps   158448 non-null  int64   
 3   avg_lat_ms   158448 non-null  int64   
 4   tests        158448 non-null  int64   
 5   devices      158448 non-null  int64   
 6   geometry     158448 non-null  geometry
 7   index_right  158448 non-null  int64   
 8   UBIGEO       158448 non-null  object  
 9   CCDD         158448 non-null  object  
 10  CCPP         158448 non-null  object  
 11  CCDI         158448 non-null  object  
 12  DEPARTAMEN   158448 non-null  object  
 13  PROVINCIA    158448 non-null  object  
 14  DISTRITO     158448 non-null  object  
 15  OBJECTID     158448 non-null  float64 
 16  ESRI_OID     158448 non-null  float64 
 17  year         158448 non-null  int64   
 

In [14]:
# Guarda el resultado como un archivo shapefile
#combined_data_1.to_file('../output/combined_data_1.shp')

C:\Users\USER\AppData\Local\Temp\ipykernel_4436\829000376.py:2: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  combined_data_1.to_file('../output/combined_data_1.shp')


In [57]:
combined_data = gp.read_file('../output/combined_data.shp') #es la misma data que

In [17]:
combined_data.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 158448 entries, 0 to 158447
Data columns (total 19 columns):
 #   Column      Non-Null Count   Dtype   
---  ------      --------------   -----   
 0   quadkey     158448 non-null  object  
 1   avg_d_kbps  158448 non-null  int64   
 2   avg_u_kbps  158448 non-null  int64   
 3   avg_lat_ms  158448 non-null  int64   
 4   tests       158448 non-null  int64   
 5   devices     158448 non-null  int64   
 6   index_righ  158448 non-null  int64   
 7   UBIGEO      158448 non-null  object  
 8   CCDD        158448 non-null  object  
 9   CCPP        158448 non-null  object  
 10  CCDI        158448 non-null  object  
 11  DEPARTAMEN  158448 non-null  object  
 12  PROVINCIA   158448 non-null  object  
 13  DISTRITO    158448 non-null  object  
 14  OBJECTID    158448 non-null  float64 
 15  ESRI_OID    158448 non-null  float64 
 16  year        158448 non-null  int64   
 17  quarter     158448 non-null  int64   
 18  geometry    1584

In [18]:
pip install folium


Note: you may need to restart the kernel to use updated packages.


In [58]:
# Filtrar los datos para quedarnos con LIMA y CALLAO
filtered_data = combined_data[combined_data['PROVINCIA'].isin(["LIMA", "CALLAO"])]

# Mostrar las primeras filas para inspeccionar
print(filtered_data.info())

<class 'geopandas.geodataframe.GeoDataFrame'>
Index: 48596 entries, 0 to 153356
Data columns (total 19 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   quadkey     48596 non-null  object  
 1   avg_d_kbps  48596 non-null  int64   
 2   avg_u_kbps  48596 non-null  int64   
 3   avg_lat_ms  48596 non-null  int64   
 4   tests       48596 non-null  int64   
 5   devices     48596 non-null  int64   
 6   index_righ  48596 non-null  int64   
 7   UBIGEO      48596 non-null  object  
 8   CCDD        48596 non-null  object  
 9   CCPP        48596 non-null  object  
 10  CCDI        48596 non-null  object  
 11  DEPARTAMEN  48596 non-null  object  
 12  PROVINCIA   48596 non-null  object  
 13  DISTRITO    48596 non-null  object  
 14  OBJECTID    48596 non-null  float64 
 15  ESRI_OID    48596 non-null  float64 
 16  year        48596 non-null  int64   
 17  quarter     48596 non-null  int64   
 18  geometry    48596 non-null  geometry
dtype

In [21]:
# Exportar el GeoDataFrame a un archivo GeoJSON
filtered_data.to_file("../output/filtered_data.geojson", driver="GeoJSON")

In [2]:
filtered_data = gp.read_file('../output/filtered_data.geojson')

In [3]:
import geopandas as gpd

In [4]:
distrital_gdf = gpd.read_file("../input/Shapefiles Lima Peru/Distritos/Distrital INEI 2023 geogpsperu SuyoPomalia.shp")

distrital_gdf.to_file("../output/Distrital INEI 2023 geogpsperu SuyoPomalia.geojson", driver="GeoJSON")

In [5]:
import geopandas as gpd

# 1. Filtrar los distritos para incluir sólo Lima y Callao
distrital_gdf_lima = distrital_gdf[distrital_gdf['PROVINCIA'].isin(['LIMA', 'CALLAO'])]

# Asegurar que ambos GeoDataFrames tienen CRS definido
if filtered_data.crs is None:
    filtered_data = filtered_data.set_crs(epsg=4326)

if distrital_gdf_lima.crs is None:
    distrital_gdf_lima = distrital_gdf_lima.set_crs(epsg=4326)

# Reproyectar a un CRS proyectado (opcional)
filtered_data = filtered_data.to_crs(epsg=32718)
distrital_gdf_lima = distrital_gdf_lima.to_crs(epsg=32718)

# 2. Calcular centroides como una columna separada
filtered_data['centroid'] = filtered_data.geometry.centroid

# 3. Crear un GeoDataFrame para los centroides
centroids_gdf = gpd.GeoDataFrame(
    filtered_data[['quadkey', 'year', 'quarter', 'centroid']],  # Usar sólo las columnas necesarias
    geometry='centroid',
    crs=filtered_data.crs
)

# 4. Realizar el spatial join con los distritos de Lima y Callao
quadkeys_with_district = gpd.sjoin(
    centroids_gdf,
    distrital_gdf_lima[['DISTRITO', 'geometry']],  # Usar sólo las columnas necesarias
    how='left',
    predicate='within'
)

# Renombrar la columna 'DISTRITO' proveniente del sjoin
quadkeys_with_district.rename(columns={'DISTRITO': 'DISTRITO_asignado'}, inplace=True)

# 5. Eliminar duplicados por quadkey, year y quarter
quadkeys_with_district = (
    quadkeys_with_district.sort_values(by=['quadkey', 'year', 'quarter'])
    .drop_duplicates(subset=['quadkey', 'year', 'quarter'], keep='first')
)

# 6. Fusionar con los datos originales y reemplazar la columna 'DISTRITO'
# Primero, eliminamos la columna 'DISTRITO' original para evitar conflictos
filtered_data = filtered_data.drop(columns=['DISTRITO'])

# Ahora hacemos el merge
quadkeys_final = filtered_data.merge(
    quadkeys_with_district[['quadkey', 'year', 'quarter', 'DISTRITO_asignado']],
    on=['quadkey', 'year', 'quarter'],
    how='left'
)

# Renombrar la columna 'DISTRITO_asignado' a 'DISTRITO'
quadkeys_final.rename(columns={'DISTRITO_asignado': 'DISTRITO'}, inplace=True)

# 7. Restaurar CRS a EPSG:4326 si es necesario
quadkeys_final = quadkeys_final.to_crs(epsg=4326)

# 8. Verificar que no queden duplicados
quadkey_district_count = (
    quadkeys_final.groupby(['quadkey', 'year', 'quarter'])['DISTRITO'].nunique()
)
quadkeys_in_multiple_districts = quadkey_district_count[quadkey_district_count >= 2]

print("Número de quadkeys únicos presentes en dos o más distritos en el mismo periodo:", len(quadkeys_in_multiple_districts))


Número de quadkeys únicos presentes en dos o más distritos en el mismo periodo: 0


In [6]:
quadkeys_final.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 48596 entries, 0 to 48595
Data columns (total 20 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   quadkey     48596 non-null  object  
 1   avg_d_kbps  48596 non-null  int64   
 2   avg_u_kbps  48596 non-null  int64   
 3   avg_lat_ms  48596 non-null  int64   
 4   tests       48596 non-null  int64   
 5   devices     48596 non-null  int64   
 6   index_righ  48596 non-null  int64   
 7   UBIGEO      48596 non-null  object  
 8   CCDD        48596 non-null  object  
 9   CCPP        48596 non-null  object  
 10  CCDI        48596 non-null  object  
 11  DEPARTAMEN  48596 non-null  object  
 12  PROVINCIA   48596 non-null  object  
 13  OBJECTID    48596 non-null  float64 
 14  ESRI_OID    48596 non-null  float64 
 15  year        48596 non-null  int64   
 16  quarter     48596 non-null  int64   
 17  geometry    48596 non-null  geometry
 18  centroid    48596 non-null  geometry
 

In [7]:
quadkeys_final_final = quadkeys_final.drop_duplicates(subset=['quadkey', 'year', 'quarter'])

In [8]:
# Contar en cuántos distritos está presente cada quadkey
quadkey_district_count = quadkeys_final_final.groupby(['quadkey'])['DISTRITO'].nunique()

# Filtrar para quadkeys presentes en dos o más distritos
quadkeys_in_multiple_districts = quadkey_district_count[quadkey_district_count >= 2]

# Contar cuántos quadkeys únicos están presentes en dos o más distritos
count_quadkeys_in_multiple_districts = len(quadkeys_in_multiple_districts)

# Mostrar el resultado
print("Número de quadkeys únicos presentes en dos o más distritos:", count_quadkeys_in_multiple_districts)
print("\nEjemplo de quadkeys presentes en múltiples distritos:")
print(quadkeys_in_multiple_districts.head())

Número de quadkeys únicos presentes en dos o más distritos: 0

Ejemplo de quadkeys presentes en múltiples distritos:
Series([], Name: DISTRITO, dtype: int64)


In [9]:
quadkeys_final_final.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Index: 36912 entries, 0 to 48595
Data columns (total 20 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   quadkey     36912 non-null  object  
 1   avg_d_kbps  36912 non-null  int64   
 2   avg_u_kbps  36912 non-null  int64   
 3   avg_lat_ms  36912 non-null  int64   
 4   tests       36912 non-null  int64   
 5   devices     36912 non-null  int64   
 6   index_righ  36912 non-null  int64   
 7   UBIGEO      36912 non-null  object  
 8   CCDD        36912 non-null  object  
 9   CCPP        36912 non-null  object  
 10  CCDI        36912 non-null  object  
 11  DEPARTAMEN  36912 non-null  object  
 12  PROVINCIA   36912 non-null  object  
 13  OBJECTID    36912 non-null  float64 
 14  ESRI_OID    36912 non-null  float64 
 15  year        36912 non-null  int64   
 16  quarter     36912 non-null  int64   
 17  geometry    36912 non-null  geometry
 18  centroid    36912 non-null  geometry
 19  D

In [10]:
quadkeys_final_final.head()

,quadkey,avg_d_kbps,avg_u_kbps,avg_lat_ms,tests,devices,index_righ,UBIGEO,CCDD,CCPP,CCDI,DEPARTAMEN,PROVINCIA,OBJECTID,ESRI_OID,year,quarter,geometry,centroid,DISTRITO
0,2100300121202033,39338,7386,18,478,56,1332,150140,15,01,40,LIMA,LIMA,1333.0,1333.0,2019,1,"POLYGON ((-76.97571 -12.09841, -76.97021 -12.0...",POINT (285266.610 8661492.114),SANTIAGO DE SURCO
1,2100300121202222,26534,6529,26,171,40,1332,150140,15,01,40,LIMA,LIMA,1333.0,1333.0,2019,1,"POLYGON ((-76.99219 -12.11989, -76.98669 -12.1...",POINT (283489.678 8659102.160),SANTIAGO DE SURCO
2,2100300120331133,23960,4519,16,225,32,1332,150140,15,01,40,LIMA,LIMA,1333.0,1333.0,2019,1,"POLYGON ((-76.99768 -12.14138, -76.99219 -12.1...",POINT (282909.038 8656721.030),SANTIAGO DE SURCO
3,2100300120313331,23637,4033,29,235,60,1332,150140,15,01,40,LIMA,LIMA,1333.0,1333.0,2019,1,"POLYGON ((-76.99768 -12.11452, -76.99219 -12.1...",POINT (282887.291 8659692.011),SANTIAGO DE SURCO
4,2100300121202231,34316,6107,17,255,42,1332,150140,15,01,40,LIMA,LIMA,1333.0,1333.0,2019,1,"POLYGON ((-76.97571 -12.11452, -76.97021 -12.1...",POINT (285279.493 8659709.406),SANTIAGO DE SURCO


In [11]:
if 'centroid' in quadkeys_final_final.columns:
    quadkeys_final_final1 = quadkeys_final_final.drop(columns=['centroid'])
    print("Columna 'centroid' eliminada.")
else:
    print("La columna 'centroid' no existe en quadkeys_final_final.")

Columna 'centroid' eliminada.


In [12]:
quadkeys_final_final1.to_file("../output/quadkeys_final_final1.geojson", driver="GeoJSON")

In [13]:
# Verificar si la columna 'centroid' existe y eliminarla
if 'centroid' in quadkeys_final_final.columns:
    quadkeys_final_final = quadkeys_final_final.drop(columns=['centroid'])
    print("Columna 'centroid' eliminada.")
else:
    print("La columna 'centroid' no existe en el GeoDataFrame.")


Columna 'centroid' eliminada.


In [14]:
# Filtrar distritos de las provincias "LIMA" y "CALLAO"
lima_callao_gdf = distrital_gdf[distrital_gdf['PROVINCIA'].isin(['LIMA', 'CALLAO'])]

# Inspeccionar los resultados
print(lima_callao_gdf.head())
print(f"Número de distritos seleccionados: {len(lima_callao_gdf)}")


     UBIGEO CCDD CCPP CCDI DEPARTAMEN PROVINCIA                    DISTRITO  \
695  070101   07   01   01     CALLAO    CALLAO                      CALLAO   
696  070102   07   01   02     CALLAO    CALLAO                  BELLAVISTA   
697  070103   07   01   03     CALLAO    CALLAO  CARMEN DE LA LEGUA REYNOSO   
698  070104   07   01   04     CALLAO    CALLAO                    LA PERLA   
699  070105   07   01   05     CALLAO    CALLAO                    LA PUNTA   

     OBJECTID  ESRI_OID                                           geometry  
695     696.0     696.0  POLYGON ((-77.12995 -11.93682, -77.12990 -11.9...  
696     697.0     697.0  POLYGON ((-77.08767 -12.05191, -77.08768 -12.0...  
697     698.0     698.0  POLYGON ((-77.08303 -12.03634, -77.08297 -12.0...  
698     699.0     699.0  POLYGON ((-77.10465 -12.06328, -77.10460 -12.0...  
699     700.0     700.0  POLYGON ((-77.15878 -12.06647, -77.15875 -12.0...  
Número de distritos seleccionados: 50


In [15]:
# Verificar el CRS actual
print(distrital_gdf.crs)

# Transformar a EPSG:4326 si es necesario
if distrital_gdf.crs != "EPSG:4326":
    lima_callao_gdf = lima_callao_gdf.to_crs(epsg=4326)


EPSG:4326


In [16]:
# Exportar a GeoJSON
lima_callao_gdf.to_file("../output/lima_callao.geojson", driver="GeoJSON")

print("Archivo GeoJSON exportado correctamente.")


Archivo GeoJSON exportado correctamente.


### quadkeys con casos de NaN en la columna DISTRITO son imputados de acuerdo al criterio de mayor área de la grilla que se intersecta con el área del distrito cercano

In [17]:
missing_distritos = quadkeys_final_final[quadkeys_final_final['DISTRITO'].isna()]
# Reproyectar a un CRS proyectado (EPSG:32718)
if missing_distritos.crs.to_string() != 'EPSG:32718':
    missing_distritos = missing_distritos.to_crs(epsg=32718)

if distrital_gdf.crs.to_string() != 'EPSG:32718':
    distrital_gdf = distrital_gdf.to_crs(epsg=32718)

# Calcular intersecciones
intersections = gpd.overlay(missing_distritos, distrital_gdf, how='intersection')

# Verificar las columnas disponibles
print("Columnas disponibles en intersections:", intersections.columns)

# Usar la columna renombrada 'DISTRITO_2' para asignar distritos
intersections['intersect_area'] = intersections.geometry.area

# Encontrar el distrito con la mayor área de intersección para cada quadkey
max_intersections = (
    intersections.loc[intersections.groupby('quadkey')['intersect_area'].idxmax()]
)

# Cambiar 'DISTRITO_2' al nombre esperado para el merge
max_intersections.rename(columns={'DISTRITO_2': 'DISTRITO'}, inplace=True)

# Asignar el distrito correspondiente
quadkeys_final_final = quadkeys_final_final.merge(
    max_intersections[['quadkey', 'DISTRITO']],
    on='quadkey',
    how='left',
    suffixes=('', '_imputed')
)

# Rellenar los valores faltantes en 'DISTRITO' con los imputados
quadkeys_final_final['DISTRITO'] = quadkeys_final_final['DISTRITO'].fillna(
    quadkeys_final_final['DISTRITO_imputed']
)

# Eliminar la columna temporal 'DISTRITO_imputed'
quadkeys_final_final = quadkeys_final_final.drop(columns=['DISTRITO_imputed'])

# Verificar el resultado final
print("Imputación completada. Número de valores faltantes restantes en 'DISTRITO':",
      quadkeys_final_final['DISTRITO'].isna().sum())

Columnas disponibles en intersections: Index(['quadkey', 'avg_d_kbps', 'avg_u_kbps', 'avg_lat_ms', 'tests', 'devices',
       'index_righ', 'UBIGEO_1', 'CCDD_1', 'CCPP_1', 'CCDI_1', 'DEPARTAMEN_1',
       'PROVINCIA_1', 'OBJECTID_1', 'ESRI_OID_1', 'year', 'quarter',
       'DISTRITO_1', 'UBIGEO_2', 'CCDD_2', 'CCPP_2', 'CCDI_2', 'DEPARTAMEN_2',
       'PROVINCIA_2', 'DISTRITO_2', 'OBJECTID_2', 'ESRI_OID_2', 'geometry'],
      dtype='object')
Imputación completada. Número de valores faltantes restantes en 'DISTRITO': 0


In [18]:
quadkeys_final_final.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 36912 entries, 0 to 36911
Data columns (total 19 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   quadkey     36912 non-null  object  
 1   avg_d_kbps  36912 non-null  int64   
 2   avg_u_kbps  36912 non-null  int64   
 3   avg_lat_ms  36912 non-null  int64   
 4   tests       36912 non-null  int64   
 5   devices     36912 non-null  int64   
 6   index_righ  36912 non-null  int64   
 7   UBIGEO      36912 non-null  object  
 8   CCDD        36912 non-null  object  
 9   CCPP        36912 non-null  object  
 10  CCDI        36912 non-null  object  
 11  DEPARTAMEN  36912 non-null  object  
 12  PROVINCIA   36912 non-null  object  
 13  OBJECTID    36912 non-null  float64 
 14  ESRI_OID    36912 non-null  float64 
 15  year        36912 non-null  int64   
 16  quarter     36912 non-null  int64   
 17  geometry    36912 non-null  geometry
 18  DISTRITO    36912 non-null  object  
d

In [28]:
quadkeys_final_final.to_file("../output/quadkeys_final_final2.geojson", driver="GeoJSON")

### Data para el análisis CSDID 

In [19]:
quadkeys_final_final['avg_d_mbps'] = quadkeys_final_final['avg_d_kbps'] / 1000

In [20]:
# Eliminar la columna 'geometry' para convertir el GeoDataFrame en un DataFrame
quadkeys_final_df = quadkeys_final_final.drop(columns=['geometry'])

# Verificar el resultado
print(quadkeys_final_df.head())
print(type(quadkeys_final_df))  # Debería ser <class 'pandas.core.frame.DataFrame'>


            quadkey  avg_d_kbps  avg_u_kbps  avg_lat_ms  tests  devices  \
0  2100300121202033       39338        7386          18    478       56   
1  2100300121202222       26534        6529          26    171       40   
2  2100300120331133       23960        4519          16    225       32   
3  2100300120313331       23637        4033          29    235       60   
4  2100300121202231       34316        6107          17    255       42   

   index_righ  UBIGEO CCDD CCPP CCDI DEPARTAMEN PROVINCIA  OBJECTID  ESRI_OID  \
0        1332  150140   15   01   40       LIMA      LIMA    1333.0    1333.0   
1        1332  150140   15   01   40       LIMA      LIMA    1333.0    1333.0   
2        1332  150140   15   01   40       LIMA      LIMA    1333.0    1333.0   
3        1332  150140   15   01   40       LIMA      LIMA    1333.0    1333.0   
4        1332  150140   15   01   40       LIMA      LIMA    1333.0    1333.0   

   year  quarter           DISTRITO  avg_d_mbps  
0  2019     

In [21]:
# Crear la columna 'DATE' combinando 'year' y 'quarter'
quadkeys_final_df['DATE'] = quadkeys_final_df['year'].astype(str) + "Q" + quadkeys_final_df['quarter'].astype(str)

# Verificar el resultado
print(quadkeys_final_df[['year', 'quarter', 'DATE']].head())


   year  quarter    DATE
0  2019        1  2019Q1
1  2019        1  2019Q1
2  2019        1  2019Q1
3  2019        1  2019Q1
4  2019        1  2019Q1


In [22]:
# Crear un diccionario de mapeo para los distritos y sus valores correspondientes
fecha_entrada_mapping = {
    "SAN MIGUEL": "2019Q4",
    "PUEBLO LIBRE": "2019Q4",
    "SURQUILLO": "2020Q3",
    "MAGDALENA DEL MAR": "2020Q3",
    "JESUS MARIA": "2020Q3",
    "LINCE": "2020Q3",
    "BREÑA": "2020Q3",
    "SANTIAGO DE SURCO": "2020Q4",
    "MIRAFLORES": "2020Q4",
    "CHORRILLOS": "2020Q4",
    "LIMA": "2020Q4",
    "SAN ISIDRO": "2020Q4",
    "CALLAO": "2020Q4",
    "LOS OLIVOS": "2020Q4",
    "LA MOLINA": "2020Q4",
    "EL AGUSTINO": "2020Q4",
    "SAN MARTIN DE PORRES": "2020Q4",
    "SAN JUAN DE LURIGANCHO": "2020Q4",
    "LA VICTORIA": "2021Q1",
    "SAN LUIS": "2021Q1",
    "BARRANCO": "2021Q1",
    "VILLA MARIA DEL TRIUNFO": "2021Q2",
    "VILLA EL SALVADOR": "2021Q2",
    "INDEPENDENCIA": "2021Q2",
    "CARMEN DE LA LEGUA REYNOSO": "2021Q2",
    "ATE": "2021Q2",
    "SAN JUAN DE MIRAFLORES": "2021Q2",
    "COMAS": "2021Q3",
    "RIMAC": "2021Q3",
    "SANTA ANITA": "2021Q3",
    "SAN BORJA": "2021Q3",
    "CARABAYLLO": "2021Q3",
    "PUENTE PIEDRA": "2021Q4"
}

# Crear la nueva columna 'Fecha_Entrada' mapeando los valores del distrito
quadkeys_final_df['Fecha_Entrada'] = quadkeys_final_df['DISTRITO'].map(fecha_entrada_mapping)

# Verificar el resultado
print(quadkeys_final_df[['DISTRITO', 'Fecha_Entrada']].head())


            DISTRITO Fecha_Entrada
0  SANTIAGO DE SURCO        2020Q4
1  SANTIAGO DE SURCO        2020Q4
2  SANTIAGO DE SURCO        2020Q4
3  SANTIAGO DE SURCO        2020Q4
4  SANTIAGO DE SURCO        2020Q4


In [23]:
quadkeys_final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36912 entries, 0 to 36911
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   quadkey        36912 non-null  object 
 1   avg_d_kbps     36912 non-null  int64  
 2   avg_u_kbps     36912 non-null  int64  
 3   avg_lat_ms     36912 non-null  int64  
 4   tests          36912 non-null  int64  
 5   devices        36912 non-null  int64  
 6   index_righ     36912 non-null  int64  
 7   UBIGEO         36912 non-null  object 
 8   CCDD           36912 non-null  object 
 9   CCPP           36912 non-null  object 
 10  CCDI           36912 non-null  object 
 11  DEPARTAMEN     36912 non-null  object 
 12  PROVINCIA      36912 non-null  object 
 13  OBJECTID       36912 non-null  float64
 14  ESRI_OID       36912 non-null  float64
 15  year           36912 non-null  int64  
 16  quarter        36912 non-null  int64  
 17  DISTRITO       36912 non-null  object 
 18  avg_d_

In [24]:
import pandas as pd

# 1. Reemplazar NaN en la columna 'Fecha_Entrada' con 0 y renombrar a 'first.treat'
quadkeys_final_df['first.treat'] = quadkeys_final_df['Fecha_Entrada'].fillna(0)

# 2. Crear la columna 'treat' donde si 'first.treat' es diferente de 0, se considera que ha recibido tratamiento
quadkeys_final_df['treat'] = quadkeys_final_df['first.treat'].apply(lambda x: 1 if x != 0 else 0)

# 3. Crear una columna numérica basada en 'first.treat'
quadkeys_final_df['first.treat_numeric'] = pd.to_numeric(quadkeys_final_df['first.treat'], errors='coerce').astype('Int64')

# 4. Crear una categoría para la columna 'DATE'
quadkeys_final_df['DATE_categorical'] = pd.Categorical(quadkeys_final_df['DATE'])

# 5. Crear un diccionario que asigne números a los trimestres
trimestre_mapping = {
    '2019Q1': 1, '2019Q2': 2, '2019Q3': 3, '2019Q4': 4,
    '2020Q1': 5, '2020Q2': 6, '2020Q3': 7, '2020Q4': 8,
    '2021Q1': 9, '2021Q2': 10, '2021Q3': 11, '2021Q4': 12,
    '2022Q1': 13, '2022Q2': 14, '2022Q3': 15, '2022Q4': 16
}

# 6. Mapear 'DATE' y 'first.treat' usando el diccionario
quadkeys_final_df['year_quarter_numeric'] = quadkeys_final_df['DATE'].map(trimestre_mapping)
quadkeys_final_df['first.treat_numeric'] = quadkeys_final_df['first.treat'].map(trimestre_mapping).fillna(0).astype('int')

# Verificar los tipos de datos
print(quadkeys_final_df[['first.treat', 'treat', 'year_quarter_numeric', 'first.treat_numeric']].dtypes)

# Verificar los primeros registros
print(quadkeys_final_df[['DATE', 'first.treat', 'treat', 'year_quarter_numeric', 'first.treat_numeric']].head())


first.treat             object
treat                    int64
year_quarter_numeric     int64
first.treat_numeric      int32
dtype: object
     DATE first.treat  treat  year_quarter_numeric  first.treat_numeric
0  2019Q1      2020Q4      1                     1                    8
1  2019Q1      2020Q4      1                     1                    8
2  2019Q1      2020Q4      1                     1                    8
3  2019Q1      2020Q4      1                     1                    8
4  2019Q1      2020Q4      1                     1                    8


In [25]:
# Calcular el porcentaje de treat y no treat
treat_counts = quadkeys_final_df['treat'].value_counts(normalize=True) * 100

# Mostrar los resultados
print("Porcentaje de treat y no treat:")
print(treat_counts)

Porcentaje de treat y no treat:
treat
1    76.183897
0    23.816103
Name: proportion, dtype: float64


In [26]:
output_csv_path = "../output/quadkeys_final_df.csv"
quadkeys_final_df.to_csv(output_csv_path, index=False)

print(f"Archivo CSV exportado en: {output_csv_path}")

Archivo CSV exportado en: ../output/quadkeys_final_df.csv


In [27]:
quadkeys_final_df

,quadkey,avg_d_kbps,avg_u_kbps,avg_lat_ms,tests,devices,index_righ,UBIGEO,CCDD,CCPP,...,quarter,DISTRITO,avg_d_mbps,DATE,Fecha_Entrada,first.treat,treat,first.treat_numeric,DATE_categorical,year_quarter_numeric
0,2100300121202033,39338,7386,18,478,56,1332,150140,15,01,...,1,SANTIAGO DE SURCO,39.338,2019Q1,2020Q4,2020Q4,1,8,2019Q1,1
1,2100300121202222,26534,6529,26,171,40,1332,150140,15,01,...,1,SANTIAGO DE SURCO,26.534,2019Q1,2020Q4,2020Q4,1,8,2019Q1,1
2,2100300120331133,23960,4519,16,225,32,1332,150140,15,01,...,1,SANTIAGO DE SURCO,23.960,2019Q1,2020Q4,2020Q4,1,8,2019Q1,1
3,2100300120313331,23637,4033,29,235,60,1332,150140,15,01,...,1,SANTIAGO DE SURCO,23.637,2019Q1,2020Q4,2020Q4,1,8,2019Q1,1
4,2100300121202231,34316,6107,17,255,42,1332,150140,15,01,...,1,SANTIAGO DE SURCO,34.316,2019Q1,2020Q4,2020Q4,1,8,2019Q1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36907,2100300123320132,67994,102462,9,1,1,1316,150124,15,01,...,4,PUCUSANA,67.994,2022Q4,NaN,0,0,0,2022Q4,16
36908,2100300123320300,82427,88467,6,37,7,1316,150124,15,01,...,4,PUCUSANA,82.427,2022Q4,NaN,0,0,0,2022Q4,16
36909,2100300123320301,144832,184148,4,1,1,1316,150124,15,01,...,4,PUCUSANA,144.832,2022Q4,NaN,0,0,0,2022Q4,16
36910,2100300123321011,10798,3021,35,24,1,1316,150124,15,01,...,4,CHILCA,10.798,2022Q4,NaN,0,0,0,2022Q4,16
